In [ ]:
import torch
torch.cuda.empty_cache()
from mlisplacement.structs import airports
from mlisplacement.structs import reasoning
from mlisplacement.extractor.chat_templates import QwenChatTemplate
from mlisplacement.extractor.reasoning import generate
from mlisplacement.utils import load_model, read_md

# from guidance import models, system, user, assistant, json as gen_json, gen
import torch
import json
import torch
from enum import Enum
import guidance
import os

In [ ]:
datadir = os.path.join("data")
filepath = os.path.join(datadir, "synthdoc_example.md")
# MODEL_ID = "Qwen/Qwen3-30B-A3B"
# MODEL_ID = "Qwen/Qwen3-30B-A3B-Instruct-2507"
MODEL_ID = "meta-llama/Llama-3.2-1B-Instruct" #This already has memory error
# MODEL_ID = "distilgpt2"

In [ ]:
hf_model, hf_tokenizer = load_model(MODEL_ID, low_cpu_mem_usage=True)
tok = hf_tokenizer
chat_template = QwenChatTemplate
model = guidance.models.Transformers(hf_model, tok, chat_template=chat_template)
charges = airports.AirportCharges()
variables = airports.AirportVariables()
input_md = read_md(filepath)

In [ ]:
import json
import traceback
import time

def generate(model, query, charge_name, markdown_content, all_charges, all_variables):
    graph_builder = ComputationGraphBuilder(model=model)

    start_time = time.perf_counter()
    llm_structured_response = graph_builder.build(
        document_content=markdown_content,
        query=query,
        charge_name=charge_name,
        all_charges=all_charges,
        all_variables=all_variables
    )
    end_time = time.perf_counter()
    build_time = end_time - start_time

    return llm_structured_response, build_time


In [ ]:
query = "Calculate the total security charge"
charge_name = "security_fee"

res_model, build_time = generate(model, query, charge_name=charge_name, markdown_content=input_md, all_charges=charges.to_dict(), all_variables=variables.to_dict())